In [1]:
from enum import Enum
from pathlib import Path
import numpy as np
import h5py
import sys 
sys.path.append('../../')

from source.helper import to_dB, print_spectogram, gen_spectogram, diff_frames, \
                          get_argmaxed_spectrogram, get_tresholded_spectogram,  \
                          get_spectrogram_metrics, plot_metrics, get_spectogram_slices


class SampleType(Enum):
    BARTEK = 0
    KUBA = 1
    OSKAR = 2
    RAFAL = 3
    FAKE = 4
    NOISE = 5


In [2]:
BASE_DIR = Path("D:/SIIUM/3 semestr/RADAR/radar-project/data")  # change to your project dir

loaded_file = h5py.File(BASE_DIR / 'complete_dataset.h5', 'r') # best to make yourself a h5 file containing complete dataset

In [3]:
data_keys =list(loaded_file['2023_05_08'].keys())
# data_keys = [k for k in data_keys if k not in ['bartek_sciana', 'wiatrak_2m_bartek_sciana']]

dict_classes = {
        'background': SampleType.NOISE,
        'bartek': SampleType.BARTEK,
        'kuba': SampleType.KUBA,
        'oddychacz_2m_o0_s100': SampleType.FAKE,
        'oddychacz_2m_o30_s0': SampleType.FAKE,
        'oddychacz_2m_o30_s100': SampleType.FAKE,
        'oddychacz_3m_o0_s100': SampleType.FAKE,
        'oddychacz_3m_o30_s0': SampleType.FAKE,
        'oddychacz_3m_o30_s100': SampleType.FAKE,
        'oddychacz_4m_o0_s100': SampleType.FAKE,
        'oddychacz_4m_o30_s0': SampleType.FAKE,
        'oddychacz_4m_o30_s100': SampleType.FAKE,
        'oskar': SampleType.OSKAR,
        'rafal': SampleType.RAFAL,
        'randomowe_chodzonko_bartek': SampleType.BARTEK,
        'randomowe_chodzonko_kuba': SampleType.KUBA,
        'randomowe_chodzonko_oskar': SampleType.OSKAR,
        'randomowe_chodzonko_rafal': SampleType.RAFAL,
        'samochod': SampleType.FAKE,
        'wiatrak_2m': SampleType.NOISE,
        'wiatrak_3m': SampleType.NOISE,
        'wiatrak_4m': SampleType.NOISE,
        'wiatrak_foliarz_2m': SampleType.NOISE,
        'wiatrak_foliarz_3m': SampleType.NOISE,
        'wiatrak_foliarz_4m': SampleType.NOISE,
}


In [6]:
slices_list = []
slices_labels = []
margin = 500

H5_FILENAME = 'slices_dataset'

try:
    h5file = h5py.File(H5_FILENAME + '.h5','r+')
except:
    print('File not exist, creating file')
    h5file = h5py.File(H5_FILENAME + '.h5','w')

for key in data_keys[:1]:
    h5group = h5file.create_group(key)

    print(key, dict_classes[key])
    sample_file = loaded_file['2023_05_08'][key]
    sample_file = sample_file[margin:-margin, 0, 0, :]
    print(sample_file.shape)
    slices_diff = []
    for d in [1,2,3,5,10,25,50,100]:
        frames_diff = diff_frames(sample_file, d)  #odejmujemy ostatnią klatkę
        diff_spect, y = gen_spectogram(frames_diff)
        diff_spect = diff_spect[:64]

        diff_spectdb = to_dB(diff_spect)
        argmax_spectrogram = get_argmaxed_spectrogram(diff_spectdb)[:,100-d:] # this returns spectrogram with 1 at argmax, else 0
        slices_diff.append(argmax_spectrogram)
    # stack slices on new axis
    slices_diff = np.stack(slices_diff, axis=2)


    print(slices_diff.shape)
    class_id = dict_classes[key].value
    slices = get_spectogram_slices(slices_diff)
    print(slices.shape)
    labels = np.full(slices.shape[0], class_id)
    print(labels.shape)
    slices_labels.append(labels)
    slices_list.append(slices)
    h5group.create_dataset('slices', data=slices)

labels = np.concatenate(slices_labels, axis=0)
con = np.concatenate(slices_list, axis=0)
print(labels.shape)
print(con.shape)
    

File not exist, creating file
background SampleType.NOISE
(13060, 512)
(64, 12960, 8)
(100, 64, 256, 8)
(100,)
(100,)
(100, 64, 256, 8)


In [11]:
import pickle
with open('data_slices_mulitchannel_newshape.pickle', 'wb') as f:
    pickle.dump((con[:4000], labels[:4000]), f)

MemoryError: 

In [12]:
# save as npy
np.save('data_slices_mulitchannel_newshape_labels.npy', labels)

In [13]:
np.save('data_slices_mulitchannel_newshape.npy', con)

In [1]:
# load npy
import numpy as np
con = np.load('data_slices_mulitchannel_newshape.npy')

In [7]:
new_h5 = h5py.File('slices_dataset.h5', 'r')


In [12]:
new_h5['background']['slices']

<HDF5 dataset "slices": shape (100, 64, 256, 8), type "<f8">